# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [5]:
import os
import sys

sys.path.append('..')
# Set your preferred name
USER = "Juncheng"
# Set the data and output directories
DATA_ROOT = '/Users/Henry/Desktop/uiuc/大二下/independent study/part 1'
OUTPUT_ROOT = '../../output1'
TRAIT = 'Osteoarthritis'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [6]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [7]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:]

['TCGA_Colon_and_Rectal_Cancer_(COADREAD)',
 'TCGA_Ocular_melanomas_(UVM)',
 'TCGA_Lower_Grade_Glioma_(LGG)',
 'CrawlData.ipynb',
 'TCGA_Mesothelioma_(MESO)',
 'TCGA_Pancreatic_Cancer_(PAAD)',
 'TCGA_Bladder_Cancer_(BLCA)',
 '.DS_Store',
 'TCGA_Kidney_Clear_Cell_Carcinoma_(KIRC)',
 'TCGA_Head_and_Neck_Cancer_(HNSC)',
 'TCGA_Lung_Cancer_(LUNG)',
 'TCGA_Ovarian_Cancer_(OV)',
 'TCGA_Testicular_Cancer_(TGCT)',
 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)',
 'TCGA_Liver_Cancer_(LIHC)',
 'TCGA_Uterine_Carcinosarcoma_(UCS)',
 'TCGA_Cervical_Cancer_(CESC)',
 'TCGA_Colon_Cancer_(COAD)',
 'TCGA_Acute_Myeloid_Leukemia_(LAML)',
 'TCGA_Kidney_Papillary_Cell_Carcinoma_(KIRP)',
 'TCGA_Endometrioid_Cancer_(UCEC)',
 'TCGA_Rectal_Cancer_(READ)',
 'TCGA_Melanoma_(SKCM)',
 'TCGA_Breast_Cancer_(BRCA)',
 'TCGA_Prostate_Cancer_(PRAD)',
 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)',
 'TCGA_Stomach_Cancer_(STAD)',
 'TCGA_Large_Bcell_Lymphoma_(DLBC)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Glioblastoma_(GBM

If no match is found, jump directly to GEO in Part 2.2

In [8]:
trait_subdir = "TCGA_Lung_Cancer_(LUNG)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LUNG.sampleMap_LUNG_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LUNG.sampleMap_HiSeqV2_PANCAN.gz')

In [9]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [10]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [11]:
clinical_data_cols[:10]

['ABSOLUTE_Ploidy',
 'ABSOLUTE_Purity',
 'AKT1',
 'ALK_translocation',
 'BRAF',
 'CBL',
 'CTNNB1',
 'Canonical_mut_in_KRAS_EGFR_ALK',
 'Cnncl_mt_n_KRAS_EGFR_ALK_RET_ROS1_BRAF_ERBB2_HRAS_NRAS_AKT1_MAP2',
 'EGFR']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [12]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['ABSOLUTE_Ploidy', 'ABSOLUTE_Purity', 'AKT1', 'ALK_translocation', 'BRAF', 'CBL', 'CTNNB1', 'Canonical_mut_in_KRAS_EGFR_ALK', 'Cnncl_mt_n_KRAS_EGFR_ALK_RET_ROS1_BRAF_ERBB2_HRAS_NRAS_AKT1_MAP2', 'EGFR', 'ERBB2', 'ERBB4', 'Estimated_allele_fraction_of_a_clonal_varnt_prsnt_t_1_cpy_pr_cll', 'Expression_Subtype', 'HRAS', 'KRAS', 'MAP2K1', 'MET', 'NRAS', 'PIK3CA', 'PTPN11', 'Pathology', 'Pathology_Updated', 'RET_translocation', 'ROS1_translocation', 'STK11', 'WGS_as_of_20120731_

In [13]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_after_initial_treatment', 'stopped_smoking_year', 'year_of_initial_pathologic_diagnosis', 'year_of_tobacco_smoking_onset']

candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [14]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [70.0, 67.0, 79.0, 68.0, 66.0],
 'days_to_birth': [-25752.0, -24532.0, -29068.0, -24868.0, -24411.0],
 'days_to_collection': [nan, nan, nan, nan, nan],
 'days_to_death': [nan, nan, '121', nan, nan],
 'days_to_initial_pathologic_diagnosis': [0.0, 0.0, 0.0, 0.0, 0.0],
 'days_to_last_followup': ['0', '1523', nan, '607', '426'],
 'days_to_new_tumor_event_after_initial_treatment': [nan,
  nan,
  nan,
  334.0,
  183.0],
 'stopped_smoking_year': [nan, nan, nan, nan, 1987.0],
 'year_of_initial_pathologic_diagnosis': [2009.0,
  2007.0,
  2007.0,
  2009.0,
  2009.0],
 'year_of_tobacco_smoking_onset': [nan, nan, nan, nan, 1963.0]}

In [15]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [16]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['MALE', 'MALE', 'FEMALE', 'MALE', 'MALE']}

In [17]:
gender_col = 'gender'

In [18]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col=age_col, gender_col=gender_col)

In [19]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [20]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Osteoarthritis,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,SLC7A10,PLA2G2C,TULP2,NPY5R,GNGT2,GNGT1,TULP3,BCL6B,GSTK1,SELP
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-05-4244-01,1,70.0,1.0,-0.056592,2.128674,-0.531035,-0.185972,-0.190878,-0.11891,-0.659794,...,3.634614,0.677218,0.015022,-1.155117,0.528367,2.75061,0.428423,-0.195527,-0.324695,0.193167
TCGA-05-4249-01,1,67.0,1.0,-0.569692,-0.318126,-0.531035,0.237428,0.011822,-0.00611,1.018406,...,4.908014,0.888518,-0.031778,-0.393117,1.632467,2.33551,-0.145577,-0.276027,-1.219395,1.850967
TCGA-05-4250-01,1,79.0,0.0,-0.899992,-0.918126,-0.531035,-0.621472,-0.454178,0.58719,-0.198894,...,0.489514,-0.086682,-0.748878,-1.173917,1.966367,-1.28139,-0.374877,0.714373,0.112305,-0.532833
TCGA-05-4382-01,1,68.0,1.0,0.667508,3.895974,-0.221535,-0.367272,0.131822,0.97629,-0.494994,...,5.730914,-0.086682,1.013722,-1.587117,0.359267,2.59051,0.806623,0.367373,-0.494595,-1.062233
TCGA-05-4384-01,1,66.0,1.0,-0.316892,2.380374,0.156865,0.232828,0.054822,0.27059,0.128606,...,4.163014,0.601218,-0.060978,-0.435417,1.111467,-1.28139,-0.128477,1.469373,0.194605,1.404667


In [21]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 1100 samples.


In [22]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Osteoarthritis', the least common label is '0' with 110 occurrences. This represents 10.00% of the dataset.
The distribution of the feature 'Osteoarthritis' in this dataset is fine.

Quartiles for 'Age':
  25%: 60.0
  50% (Median): 67.0
  75%: 73.0
Min: 38.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 443 occurrences. This represents 40.27% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [23]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [24]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [25]:
dataset = 'GEO'
trait_subdir = "Osteoarthritis"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE141934', 'GSE137634', 'GSE142049', 'GSE236924']

Repeat the below steps for all the accession numbers

In [26]:
cohort = accession_num = "GSE141934"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/Users/Henry/Desktop/uiuc/大二下/independent study/part 1/GEO/Osteoarthritis/GSE141934/GSE141934_family.soft.gz',
 '/Users/Henry/Desktop/uiuc/大二下/independent study/part 1/GEO/Osteoarthritis/GSE141934/GSE141934_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [27]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Transcriptional data of inflamatory arthritis T cells."
!Series_summary	"With a focus on rheumatoid arthritis (RA), we sought new insight into genetic mechanisms of adaptive immune dysregulation to help prioritise molecular pathways for targeting in this and related immune pathologies. Whole genome methylation and transcriptional data from isolated CD4+ T cells and B cells of >100 genotyped and phenotyped inflammatory arthritis patients, all of whom were naïve to immunomodulatory treatments, were obtained. Analysis integrated these comprehensive data with GWAS findings across IMDs and other publically available resources."
!Series_overall_design	"Suspected inflammatory arthritis patients of Northern European ancestry were recruited prior to treatment with immunomodulatory drugs. RA patients were classified using current, internationally accepted criteria, and matched with disease controls in respect of demographic and clinical characteristics. CD4+ cells were isolated fr

In [28]:
clinical_data.head()

,!Sample_geo_accession,GSM4216498,GSM4216499,GSM4216500,GSM4216501,GSM4216502,GSM4216503,GSM4216504,GSM4216505,GSM4216506,...,GSM4216588,GSM4216589,GSM4216590,GSM4216591,GSM4216592,GSM4216593,GSM4216594,GSM4216595,GSM4216596,GSM4216597
0,!Sample_characteristics_ch1,patient: 1072,patient: 1085,patient: 1076,patient: 1087,patient: 1080,patient: 1088,patient: 1083,patient: 1094,patient: 1050,...,patient: 929,patient: 945,patient: 946,patient: 975,patient: 978,patient: 980,patient: 967,patient: 973,patient: 992,patient: 974
1,!Sample_characteristics_ch1,gender: F,gender: F,gender: F,gender: M,gender: F,gender: F,gender: F,gender: M,gender: M,...,gender: F,gender: M,gender: M,gender: F,gender: F,gender: F,gender: F,gender: F,gender: F,gender: F
2,!Sample_characteristics_ch1,age: 50,age: 43,age: 66,age: 55,age: 52,age: 54,age: 63,age: 61,age: 58,...,age: 48,age: 61,age: 43,age: 54,age: 62,age: 20,age: 62,age: 50,age: 60,age: 69
3,!Sample_characteristics_ch1,tissue: peripheral blood,tissue: peripheral blood,tissue: peripheral blood,tissue: peripheral blood,tissue: peripheral blood,tissue: peripheral blood,tissue: peripheral blood,tissue: peripheral blood,tissue: peripheral blood,...,tissue: peripheral blood,tissue: peripheral blood,tissue: peripheral blood,tissue: peripheral blood,tissue: peripheral blood,tissue: peripheral blood,tissue: peripheral blood,tissue: peripheral blood,tissue: peripheral blood,tissue: peripheral blood
4,!Sample_characteristics_ch1,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,...,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells


In [29]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['patient: 1072',
  'patient: 1085',
  'patient: 1076',
  'patient: 1087',
  'patient: 1080',
  'patient: 1088',
  'patient: 1083',
  'patient: 1094',
  'patient: 1050',
  'patient: 1067',
  'patient: 1051',
  'patient: 1054',
  'patient: 1070',
  'patient: 1058',
  'patient: 2010',
  'patient: 2012',
  'patient: 2029',
  'patient: 2075',
  'patient: 2062',
  'patient: 2078',
  'patient: 2086',
  'patient: 2087',
  'patient: 2067',
  'patient: 2072',
  'patient: 2090',
  'patient: 1019',
  'patient: 1020',
  'patient: 1003',
  'patient: 1008',
  'patient: 2030'],
 1: ['gender: F', 'gender: M'],
 2: ['age: 50',
  'age: 43',
  'age: 66',
  'age: 55',
  'age: 52',
  'age: 54',
  'age: 63',
  'age: 61',
  'age: 58',
  'age: 79',
  'age: 69',
  'age: 57',
  'age: 46',
  'age: 44',
  'age: 59',
  'age: 81',
  'age: 60',
  'age: 92',
  'age: 45',
  'age: 47',
  'age: 27',
  'age: 38',
  'age: 51',
  'age: 70',
  'age: 56',
  'age: 53',
  'age: 74',
  'age: 49',
  'age: 31',
  'age: 65'],


Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [30]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Osteoarthritis\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Osteoarthritis\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [31]:
is_gene_availabe = True
trait_row = 6
age_row = 2
gender_row = 1

trait_type = 'binary'

In [32]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [42]:
def convert_trait(tissue_type):
    """
    Convert tissue type to epilepsy presence (binary).
    Assuming epilepsy presence for 'Hippocampus' tissue.
    """
    if tissue_type == 'tissue: Hippocampus':
        return 1  # Epilepsy present
    else:
        return 0  # Epilepsy not present

def convert_age(age_string):
    """
    Convert age string to a numerical value.
    """
    if age_string.startswith('age: '):
        try:
            age = age_string.split('age: ')[1].rstrip('') # Extracting numeric part
            if age.endswith('y'):
                age = age.rstrip('y')
            return int(age)
        except ValueError:
            return None  # In case of parsing error
    else:
        return None  # If age string does not follow expected format

def convert_gender(gender_string):
    """
    Convert gender string to a binary representation.
    Female -> 0, Male -> 1, Unknown -> None
    """
    gender = gender_string.split(':')[1].strip(' ')
    if gender == 'F':
        return 0
    elif gender == 'M':
        return 1
    else:
        return None  # In case of unknown gender

# Example usage
example_tissue = 'tissue: Hippocampus'
example_age = 'age: 23y'
example_gender = 'gender: M'

converted_trait = convert_trait(example_tissue)
converted_age = convert_age(example_age)
converted_gender = convert_gender(example_gender)

converted_trait, converted_age, converted_gender

(1, 23, 1)

In [43]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

,GSM4216498,GSM4216499,GSM4216500,GSM4216501,GSM4216502,GSM4216503,GSM4216504,GSM4216505,GSM4216506,GSM4216507,...,GSM4216588,GSM4216589,GSM4216590,GSM4216591,GSM4216592,GSM4216593,GSM4216594,GSM4216595,GSM4216596,GSM4216597
Osteoarthritis,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Age,50,43,66,55,52,54,63,61,58,79,...,48,61,43,54,62,20,62,50,60,69
Gender,0,0,0,1,0,0,0,1,1,1,...,0,1,1,0,0,0,0,0,0,0


### Genetic data preprocessing and final filtering

In [44]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM4216498,GSM4216499,GSM4216500,GSM4216501,GSM4216502,GSM4216503,GSM4216504,GSM4216505,GSM4216506,GSM4216507,...,GSM4216588,GSM4216589,GSM4216590,GSM4216591,GSM4216592,GSM4216593,GSM4216594,GSM4216595,GSM4216596,GSM4216597
ID,,,,,,,,,,,,,,,,,,,,,
ILMN_1343291,14.539987,14.405712,14.448393,14.499786,14.312660,14.607415,14.223938,14.492985,14.522590,14.492985,...,14.487565,14.461486,14.644446,14.567983,14.522590,14.505918,14.444680,14.522590,14.512663,14.539987
ILMN_1343295,11.609760,11.741243,11.371202,11.798545,11.574261,11.569844,11.648688,11.811691,11.648688,11.593766,...,11.555161,11.745084,11.926882,11.457803,11.675655,11.845560,11.882650,11.385035,11.280602,11.378086
ILMN_1651228,14.140474,13.815123,13.920783,13.842807,13.826392,13.745337,13.947525,13.820681,13.735698,13.853578,...,13.791062,13.764439,13.882919,13.949832,14.055596,13.983177,13.992954,13.937582,13.687524,14.015118
ILMN_1651229,6.664432,6.336034,6.577622,6.594562,6.424703,6.393298,6.774854,6.437850,6.240510,6.429385,...,6.596588,6.436289,6.534840,6.450617,6.402912,6.640206,6.533076,6.035299,6.298384,5.883456
ILMN_1651254,10.640088,10.756908,10.633290,10.372284,10.055662,10.446756,10.183049,10.311278,10.751252,10.351995,...,9.851118,9.772957,8.797985,9.998293,9.872110,9.692748,9.220622,9.683137,10.617125,9.603272


In [45]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['ILMN_1343291',
 'ILMN_1343295',
 'ILMN_1651228',
 'ILMN_1651229',
 'ILMN_1651254',
 'ILMN_1651259',
 'ILMN_1651262',
 'ILMN_1651278',
 'ILMN_1651315',
 'ILMN_1651316',
 'ILMN_1651341',
 'ILMN_1651343',
 'ILMN_1651347',
 'ILMN_1651378',
 'ILMN_1651385',
 'ILMN_1651403',
 'ILMN_1651405',
 'ILMN_1651429',
 'ILMN_1651433',
 'ILMN_1651438']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [46]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651228', 'ILMN_1651229', 'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651262', 'ILMN_1651278', 'ILMN_1651315', 'ILMN_1651316', 'ILMN_1651341', 'ILMN_1651343', 'ILMN_1651347', 'ILMN_1651378', 'ILMN_1651385', 'ILMN_1651403', 'ILMN_1651405', 'ILMN_1651429', 'ILMN_1651433', 'ILMN_1651438']\n"

If not required, jump directly to the gene normalization step

In [51]:
requires_gene_mapping = False

In [52]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    print(gene_annotation)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [53]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')

In [54]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'ENTREZ_GENE_ID'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [55]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [56]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 154 samples.


In [57]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Osteoarthritis', the least common label is '0.0' with 100 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Osteoarthritis' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 49.75
  50% (Median): 56.0
  75%: 65.25
Min: 20.0
Max: 92.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 31 occurrences. This represents 31.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



True

In [58]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')

In [59]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [60]:
from utils import *

In [61]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_age')
best_cohort

'Xena'

In [62]:
ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,Xena,True,True,False,True,True,1100,


In [63]:
ck=100
listq=[]
for df in pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'),chunksize=ck):
    listq.append(df)
merged_data = pd.concat(listq)
#merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'))

In [64]:
merged_data.head()

,Osteoarthritis,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,SLC7A10,PLA2G2C,TULP2,NPY5R,GNGT2,GNGT1,TULP3,BCL6B,GSTK1,SELP
0,1,70.0,1.0,-0.056592,2.128674,-0.531035,-0.185972,-0.190878,-0.11891,-0.659794,...,3.634614,0.677218,0.015022,-1.155117,0.528367,2.75061,0.428423,-0.195527,-0.324695,0.193167
1,1,67.0,1.0,-0.569692,-0.318126,-0.531035,0.237428,0.011822,-0.00611,1.018406,...,4.908014,0.888518,-0.031778,-0.393117,1.632467,2.33551,-0.145577,-0.276027,-1.219395,1.850967
2,1,79.0,0.0,-0.899992,-0.918126,-0.531035,-0.621472,-0.454178,0.58719,-0.198894,...,0.489514,-0.086682,-0.748878,-1.173917,1.966367,-1.28139,-0.374877,0.714373,0.112305,-0.532833
3,1,68.0,1.0,0.667508,3.895974,-0.221535,-0.367272,0.131822,0.97629,-0.494994,...,5.730914,-0.086682,1.013722,-1.587117,0.359267,2.59051,0.806623,0.367373,-0.494595,-1.062233
4,1,66.0,1.0,-0.316892,2.380374,0.156865,0.232828,0.054822,0.27059,0.128606,...,4.163014,0.601218,-0.060978,-0.435417,1.111467,-1.28139,-0.128477,1.469373,0.194605,1.404667


In [65]:
# If both age and gender features are available, select 'age' as the condition.
condition = 'Age'
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

In [66]:
X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [67]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [68]:

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [69]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
The cross-validation accuracy is 56.09% ± 5.47%


In [70]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

/Users/Henry/Library/Python/3.7/lib/python/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in true_divide
  ts = beta / np.sqrt(var * sigma)
/Users/Henry/Library/Python/3.7/lib/python/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in true_divide
  ts = beta / np.sqrt(var * sigma)
/Users/Henry/Library/Python/3.7/lib/python/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in true_divide
  ts = beta / np.sqrt(var * sigma)
/Users/Henry/Library/Python/3.7/lib/python/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in true_divide
  ts = beta / np.sqrt(var * sigma)
/Users/Henry/Library/Python/3.7/lib/python/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in true_divide
  ts = beta / np.sqrt(var * sigma)


The cross-validation accuracy is 89.45% ± 2.91%


In [71]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

alpha for Lasso: 0.0003


/Users/Henry/Library/Python/3.7/lib/python/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in true_divide
  ts = beta / np.sqrt(var * sigma)


### 5. Discussion and report

In [72]:

feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: -0.0077
Found 19 genes with non-zero coefficients associated with the trait 'Osteoarthritis' conditional on the factor 'Age'. These genes are identified as significant based on the regression model.
Variable  Coefficient  Absolute Coefficient
  CT47A7     0.774364              0.774364
     BPI     0.715783              0.715783
    SOX1     0.649089              0.649089
  IFIT1B    -0.453567              0.453567
   SNHG5     0.401117              0.401117
   SPAM1    -0.331380              0.331380
   OR4D6     0.315670              0.315670
    DYTN    -0.270333              0.270333
  CLEC3A    -0.167602              0.167602
TBC1D3P2    -0.115997              0.115997
   ADAD1    -0.113473              0.113473
C10orf53     0.095358              0.095358
  OR4F17     0.093556              0.093556
  CELA2A    -0.084290              0.084290
     ALK    -0.080820              0.080820
  IFITM5     0.075685 

/Users/Henry/Desktop/uiuc/大二下/independent study/part 1/utils.py:486: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df['Absolute Coefficient'] = gene_regression_df['Coefficient'].abs()


In [73]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)

Effect of the condition on the target variable:
Variable: Age
Coefficient: -0.0077
Found 14 significant genes associated with the trait 'Osteoarthritis' conditional on the factor 'Age', with corrected p-value < 0.05:
Variable  Coefficient      p_value  corrected_p_value
   MEX3A     0.061067 2.141278e-07           0.003713
 SLC19A1    -0.038906 6.275702e-07           0.005441
  CRABP2     0.036464 1.612588e-06           0.009320
   HMGB3     0.042605 8.395292e-06           0.020795
 DEFB118     0.028302 7.094742e-06           0.020795
     IL3    -0.016963 5.099702e-06           0.020795
  FAM83A     0.043837 8.078703e-06           0.020795
    RCC1     0.041264 1.790761e-05           0.028227
    NUF2     0.068381 1.766368e-05           0.028227
    SPP1     0.032008 1.786760e-05           0.028227
    ETV4     0.037379 1.674869e-05           0.028227
   EPB42    -0.020446 2.158322e-05           0.031186
   CENPM    -0.051855 3.055404e-05           0.040752
 PLEKHN1     0.044351 3.348

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
